In [9]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw, ImageFont
import requests
import random


c:\Users\yunus.yesil\Desktop\python-projects\Image_Detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
url2 = "https://img-s1.onedio.com/id-558909bed2a15480349e158e/rev-0/w-635/listing/f-jpg-webp/s-7d14de6baa50408719ec71b4ee671a2961a4f81f.webp"
# image = Image.open(requests.get(url2, stream=True).raw)

image = Image.open("cat_img.jpg")

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


Detected remote with confidence 0.998 at location [40.16, 70.81, 175.55, 117.98]
Detected remote with confidence 0.996 at location [333.24, 72.55, 368.33, 187.66]
Detected couch with confidence 0.995 at location [-0.02, 1.15, 639.73, 473.76]
Detected cat with confidence 0.999 at location [13.24, 52.05, 314.02, 470.93]
Detected cat with confidence 0.999 at location [345.4, 23.85, 640.37, 368.72]


In [13]:
draw = ImageDraw.Draw(image)

# Define a color mapping dictionary to assign unique colors to each class label
color_map = {}

# Specify font size
font_size = 16
font = ImageFont.truetype("arial.ttf", font_size)

# Specify line width
line_width = 2

img_name = ""
k=0

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    
    box = [round(i, 2) for i in box.tolist()]
    class_label = model.config.id2label[label.item()]
    confidence = round(score.item(), 3)

    # Generate a random color for the class label if it doesn't exist in the color map
    if class_label not in color_map:
        color_map[class_label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    # Draw rectangle with the color corresponding to the class label
    draw.rectangle(box, outline=color_map[class_label], width=line_width)

    # Add text with label and confidence, using the same color as the rectangle
    text = f"{class_label}: {confidence}"
    
    # Create img name
    if k < 4 and class_label not in img_name :
        img_name += "_"+class_label

    draw.text((box[0], box[1]), text, fill=color_map[class_label], font=font)
    k += 1

image.show()
image.save(f"img_with{img_name}.png")